# Explore configuration file json 

# Create storage object to store the network in

In [1]:
from bsb.storage import Storage
from bsb.config import from_json
from bsb.core import Scaffold
from bsb.morphologies import Morphology
import h5py    

# Load a configuration from file
config = from_json("netAtlasConfig.json")
# Create storage object to store the network in
storage = Storage("hdf5", "VISp.hdf5")
network = Scaffold(config, storage)


# Explore the region of interest (here primary visual cortex as example)
properties from allen atlas and density (nrrd data) from blue brain 

In [2]:
from bsb.topology.partition import NrrdVoxels, AllenStructure
import numpy as np
import nrrd
#voxels = NrrdVoxels(source="annotation_25.nrrd", voxel_size=25)
#vs = voxels.get_voxelset()
#print(vs)
#print(len(vs.data))
#print("mouse brain volume [mm3]: ", 25*25*25*len(vs.data)*10**(-9))


struct = "VISp"
voxel_size=25

# Retrieve the raw Allen information on a structure:
node = AllenStructure.find_structure(struct)
print("Allen node:", node["name"])

# Get all of the IDs and names that are part of this structure:
ids = AllenStructure.get_structure_idset(struct)
names_subregions=[]
for i in ids:
    #print(i)
    name = AllenStructure.find_structure(int(i))
    names_subregions.append(name["name"])
print("subregions: ", names_subregions)

# Get the boolean mask of the structure. 1's are part of the structure, 0s aren't.
mask = AllenStructure.get_structure_mask(struct)
#print(mask.shape)
print("The structure contains", np.sum(mask), "voxels")
print("struct volume [mm3]: ", voxel_size**3*np.sum(mask)/10**9)
# You can use this to mask other images of the brain, such as a fictitious density file:
brain_image, _ = nrrd.read("data/inh_density.nrrd")
struct_image = np.where(mask, brain_image, np.nan)
# Or an array of the values:
struct_values = brain_image[mask]
#print("raw number of inh neurons: ", np.sum(struct_values)) with the nrrd of number of neurons / voxel
print("raw number of inh neurons: ", np.nanmean(struct_values)*voxel_size**3*np.sum(mask))

brain_image, _ = nrrd.read("data/exc_density.nrrd")
struct_values = brain_image[mask]
print("raw number of exc neurons: ", np.nanmean(struct_values)*voxel_size**3*np.sum(mask))

# Saving morphologies in the network

Morphologies can come from `.asc` or `.swc` files. In this notebook, the morphologies are available in the `morphologies/` folder, and save them in our network:

In [3]:
network.morphologies.save("PyramidalCell", Morphology.from_swc("morphologies/PyramidalCell.swc"))
network.morphologies.save("StellateCell_v1",  Morphology.from_swc("morphologies/StellateCell_v1.swc"))
network.morphologies.save("StellateCell_v2",  Morphology.from_swc("morphologies/StellateCell_v2.swc"))

# plot one of the cell morphologies
from bsb.plotting import plot_morphology
cell = network.morphologies.load("PyramidalCell")
plot_morphology(cell);

from bsb.plotting import plot_morphology
cell = network.morphologies.load("StellateCell_v1")
plot_morphology(cell);

from bsb.plotting import plot_morphology
cell = network.morphologies.load("StellateCell_v2")
plot_morphology(cell);

# Compile (reconstruction) the network 
(here only the placement is defined,  not the connectivity)
(as indicated in the .json, here only 2 layers are filled with excitatory and inhibitory neurons)

In [4]:
import bsb.options
bsb.options.verbosity = 4
network.compile(clear=True)

# Plot the 3D network with placed neurons 

In [5]:
from bsb.plotting import plot_network
plot_network(network);

# Another option to create the own project on bsb

In [ ]:
!bsb new my_first_model --quickstart